In [1]:
from deepcrypto.backtest import *
from deepcrypto.portfolio_analysis import *
from deepcrypto.data_utils.crawlers.bitfinex_kaggle import *
from talib.abstract import *

In [2]:
def strategy(df, config):
    mean_volume = df["volume"].rolling(int(config["mean_volume_period"])).mean().shift(1)
    volume_signal = (df["volume"] / mean_volume) > config["volume_times"]
    
    returns = df["close"].pct_change(int(config["return_period"]))
    return_signal = ((returns == returns.rolling(int(config["return_signal_period"])).max()).rolling(int(config["any_return_period"])).max() == 1)
        
    breakout_signal = df["high"] > df["high"].rolling(int(config["time_cut"] // 2)).max().shift(1)
    breakout_signal_short = df["low"] < df["low"].rolling(int(config["time_cut"] // 2)).min().shift(1)
    
    ma = df["close"].rolling(int(config["ma_period"])).mean()
    ma_signal = ma.diff() > 0
    
    df["enter_long"] = return_signal & volume_signal & ma_signal & breakout_signal
    df["bet"] = np.clip(0.01 / df.close.pct_change().rolling(200).std(), 0.1, 2) * 2
    df["time_cut"] = config["time_cut"]
    df["stop_loss"] = config["stop_loss"] / df["bet"]
    df["take_profit"] = config["take_profit"]
    df["trade_cost"] = 0.001
    

    
    return df

In [3]:
def backtest(config, ticker):
    data = data.backtest.add_defaults()

    order, port = strategy(data, config).backtest()
    qs.reports.full(port["portfolio_value"].resample("1D").last(), benchmark=port["open"].resample("1D").last())

In [4]:
import ray
from deepcrypto.optimization import *

def metric_fn(order_df, port_df):
    return {"sharpe" : qs.stats.sharpe(port_df["portfolio_value"].resample("1D").last())}

def optimize(df, ticker):
    optimizer = BruteForceOptimizer(
        data=df,
        strategy=strategy,
        config_dict = {
            "time_cut" : [12, 24, 48, 96],
            "stop_loss" : [0.02, 0.03, 0.05, 0.1],
            "take_profit" : [0.05, 0.07, 0.1, 0.15, 0.2],
            "return_signal_period" : ([12, 24, 48, 96]),
            "ma_period" : [12, 24, 48, 96],
            "mean_volume_period" : [6, 12, 24, 48, 96],
            "return_period" : [6, 12, 24, 48, 96],
            "any_return_period" : [3],
            "ma_period" : [6, 12, 24, 48, 96],
            "volume_times" : [3, 4, 5, 6, 7],
        },
        metric_fn=metric_fn,
        result_dir="./results",
        strategy_name=f"VolumeBreakout_{ticker}",
        n_cores=12
    )
    optimizer.optimize()
    optimizer.save_results()
    return optimizer


/home/ych/anaconda3/envs/DeepCrypto/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


In [5]:
ray.init()

2021-05-28 12:19:39,219	INFO services.py:1269 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.68.106',
 'raylet_ip_address': '192.168.68.106',
 'redis_address': '192.168.68.106:6379',
 'object_store_address': '/tmp/ray/session_2021-05-28_12-19-38_256386_2763/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-05-28_12-19-38_256386_2763/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-05-28_12-19-38_256386_2763',
 'metrics_export_port': 57916,
 'node_id': '13976716e5bbc14fc94b4fc02ce8eee64cf4cd6789995e98adca7745'}

In [ ]:
data = read_binance_data("/home/ych/Storage/binance/binance.db", "1H", "BTCUSDT")
data = data.backtest.add_defaults()

data = data[:int(data.index.__len__() * 0.7)]

optimizer = optimize(data, "BTCUSDT")
config = optimizer.get_best_result("sharpe")

 10%|█         | 1708/16667 [05:45<55:12,  4.52it/s]  

In [ ]:
pprint(config)

In [ ]:
backtest_and_report(config, "BTCUSDT")